In [1]:
import keras
from tensorflow.keras.utils import load_img, img_to_array
from keras.models import Model
from keras.applications import imagenet_utils
from keras.applications import MobileNet
from keras.applications.mobilenet import preprocess_input
import numpy as np
from IPython.display import Image

In [6]:
myModel = keras.applications.mobilenet.MobileNet(input_shape=(224,224,3),weights='imagenet',include_top=False)

In [7]:
myModel.summary()

Model: "mobilenet_1.00_224"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 conv1 (Conv2D)              (None, 112, 112, 32)      864       
                                                                 
 conv1_bn (BatchNormalizatio  (None, 112, 112, 32)     128       
 n)                                                              
                                                                 
 conv1_relu (ReLU)           (None, 112, 112, 32)      0         
                                                                 
 conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)     288       
                                                                 
 conv_dw_1_bn (BatchNormaliz  (None, 112, 112, 32)     128       
 ation)                                         

                                                                 
 conv_pw_8_bn (BatchNormaliz  (None, 14, 14, 512)      2048      
 ation)                                                          
                                                                 
 conv_pw_8_relu (ReLU)       (None, 14, 14, 512)       0         
                                                                 
 conv_dw_9 (DepthwiseConv2D)  (None, 14, 14, 512)      4608      
                                                                 
 conv_dw_9_bn (BatchNormaliz  (None, 14, 14, 512)      2048      
 ation)                                                          
                                                                 
 conv_dw_9_relu (ReLU)       (None, 14, 14, 512)       0         
                                                                 
 conv_pw_9 (Conv2D)          (None, 14, 14, 512)       262144    
                                                                 
 conv_pw_9

In [8]:
from keras.layers import Dense,GlobalAveragePooling2D

x=myModel.output
x=GlobalAveragePooling2D()(x)
x=Dense(1024,activation='relu')(x) 
x=Dense(512,activation='relu')(x) 
y=Dense(3,activation='softmax')(x) 

myNewModel=Model(inputs=myModel.input,outputs=y)

In [9]:
myNewModel.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 conv1 (Conv2D)              (None, 112, 112, 32)      864       
                                                                 
 conv1_bn (BatchNormalizatio  (None, 112, 112, 32)     128       
 n)                                                              
                                                                 
 conv1_relu (ReLU)           (None, 112, 112, 32)      0         
                                                                 
 conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)     288       
                                                                 
 conv_dw_1_bn (BatchNormaliz  (None, 112, 112, 32)     128       
 ation)                                                      

                                                                 
 conv_pw_8_bn (BatchNormaliz  (None, 14, 14, 512)      2048      
 ation)                                                          
                                                                 
 conv_pw_8_relu (ReLU)       (None, 14, 14, 512)       0         
                                                                 
 conv_dw_9 (DepthwiseConv2D)  (None, 14, 14, 512)      4608      
                                                                 
 conv_dw_9_bn (BatchNormaliz  (None, 14, 14, 512)      2048      
 ation)                                                          
                                                                 
 conv_dw_9_relu (ReLU)       (None, 14, 14, 512)       0         
                                                                 
 conv_pw_9 (Conv2D)          (None, 14, 14, 512)       262144    
                                                                 
 conv_pw_9

In [10]:
for i,layer in enumerate(myNewModel.layers):
  print(i,layer.name)

0 input_3
1 conv1
2 conv1_bn
3 conv1_relu
4 conv_dw_1
5 conv_dw_1_bn
6 conv_dw_1_relu
7 conv_pw_1
8 conv_pw_1_bn
9 conv_pw_1_relu
10 conv_pad_2
11 conv_dw_2
12 conv_dw_2_bn
13 conv_dw_2_relu
14 conv_pw_2
15 conv_pw_2_bn
16 conv_pw_2_relu
17 conv_dw_3
18 conv_dw_3_bn
19 conv_dw_3_relu
20 conv_pw_3
21 conv_pw_3_bn
22 conv_pw_3_relu
23 conv_pad_4
24 conv_dw_4
25 conv_dw_4_bn
26 conv_dw_4_relu
27 conv_pw_4
28 conv_pw_4_bn
29 conv_pw_4_relu
30 conv_dw_5
31 conv_dw_5_bn
32 conv_dw_5_relu
33 conv_pw_5
34 conv_pw_5_bn
35 conv_pw_5_relu
36 conv_pad_6
37 conv_dw_6
38 conv_dw_6_bn
39 conv_dw_6_relu
40 conv_pw_6
41 conv_pw_6_bn
42 conv_pw_6_relu
43 conv_dw_7
44 conv_dw_7_bn
45 conv_dw_7_relu
46 conv_pw_7
47 conv_pw_7_bn
48 conv_pw_7_relu
49 conv_dw_8
50 conv_dw_8_bn
51 conv_dw_8_relu
52 conv_pw_8
53 conv_pw_8_bn
54 conv_pw_8_relu
55 conv_dw_9
56 conv_dw_9_bn
57 conv_dw_9_relu
58 conv_pw_9
59 conv_pw_9_bn
60 conv_pw_9_relu
61 conv_dw_10
62 conv_dw_10_bn
63 conv_dw_10_relu
64 conv_pw_10
65 conv_pw_1

In [11]:
for layer in myNewModel.layers[:86]: # layer 0 sampai dengan 85 non trainable
    layer.trainable=False
for layer in myNewModel.layers[86:]: # layer 86 sampai dengan terakhir
    layer.trainable=True

In [12]:
from keras.preprocessing.image import ImageDataGenerator


train_set=ImageDataGenerator(preprocessing_function=preprocess_input)

train_set2=train_set.flow_from_directory(
    directory='data/train',
    target_size=(224,224),
    color_mode='rgb',
    batch_size=10,
    class_mode='categorical',
    shuffle=True)

Found 130 images belonging to 3 classes.


In [13]:
val_set=ImageDataGenerator(preprocessing_function=preprocess_input)

val_set2=val_set.flow_from_directory(
    directory='data/val',
    target_size=(224,224),
    color_mode='rgb',
    batch_size=10,
    class_mode='categorical',
    shuffle=True)

Found 15 images belonging to 3 classes.


In [14]:
opt = keras.optimizers.Adam(learning_rate=0.001)
myNewModel.compile(optimizer=opt,loss="categorical_crossentropy",metrics=['accuracy'])
myNewModel.fit(train_set2,epochs=20,batch_size=10,validation_data=val_set2)

Epoch 1/20
 5/13 [==========>...................] - ETA: 2s - loss: 1.7860 - accuracy: 0.4800

C:\Users\nemue\anaconda3\lib\site-packages\PIL\Image.py:945: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


13/13 [==============================] - 7s 440ms/step - loss: 1.3225 - accuracy: 0.6385 - val_loss: 0.2180 - val_accuracy: 0.9333
Epoch 2/20
13/13 [==============================] - 4s 324ms/step - loss: 0.3134 - accuracy: 0.9077 - val_loss: 0.1740 - val_accuracy: 0.9333
Epoch 3/20
13/13 [==============================] - 4s 345ms/step - loss: 0.0863 - accuracy: 0.9769 - val_loss: 0.3319 - val_accuracy: 0.9333
Epoch 4/20
13/13 [==============================] - 4s 338ms/step - loss: 0.0239 - accuracy: 1.0000 - val_loss: 0.3856 - val_accuracy: 0.9333
Epoch 5/20
13/13 [==============================] - 4s 336ms/step - loss: 0.0035 - accuracy: 1.0000 - val_loss: 0.2365 - val_accuracy: 0.9333
Epoch 6/20
13/13 [==============================] - 4s 329ms/step - loss: 0.0016 - accuracy: 1.0000 - val_loss: 0.2740 - val_accuracy: 0.9333
Epoch 7/20
13/13 [==============================] - 4s 343ms/step - loss: 6.8701e-04 - accuracy: 1.0000 - val_loss: 0.2901 - val_accuracy: 0.9333
Epoch 8/20
13

In [24]:
img = keras.utils.load_img('images.jfif', target_size=(224, 224))
img_array = keras.utils.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)
gambar = keras.applications.mobilenet.preprocess_input(img_array)

In [25]:
predictions = myNewModel.predict(gambar)

1/1 [==============================] - 0s 47ms/step


In [26]:
np.round(predictions)

array([[0., 1., 0.]], dtype=float32)

In [27]:
myNewModel.save('myNewModel.h5')